In [1]:
!curl https://course.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   322  100   322    0     0   1229      0 --:--:-- --:--:-- --:--:--  1229
Updating fastai...
Done.


In [2]:
import os
import gc
import re

import cv2
import math
import numpy as np
import scipy as sp
import pandas as pd

import tensorflow as tf

from keras.utils import plot_model
import tensorflow.keras.layers as L
from keras.utils import model_to_dot
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model

from tensorflow.keras.applications import InceptionResNetV2,ResNet50

import seaborn as sns
from tqdm import tqdm
import matplotlib.cm as cm
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

tqdm.pandas()
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

np.random.seed(0)
tf.random.set_seed(0)

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from zipfile import ZipFile 
def unZip(file_name):
  with ZipFile(file_name, 'r') as zip: 
      zip.extractall() 
      print('Done!') 

In [5]:
unZip('drive/My Drive/Research/dataset_tb.zip')

Done!


In [6]:
!ls dataset

0  1


In [0]:
import glob
filesN= sorted(glob.glob('dataset/0/*.png'))
filesP= sorted(glob.glob('dataset/1/*.png'))

In [8]:
filesN,filesP

(['dataset/0/CHNCXR_0001_0.png',
  'dataset/0/CHNCXR_0002_0.png',
  'dataset/0/CHNCXR_0003_0.png',
  'dataset/0/CHNCXR_0004_0.png',
  'dataset/0/CHNCXR_0005_0.png',
  'dataset/0/CHNCXR_0006_0.png',
  'dataset/0/CHNCXR_0007_0.png',
  'dataset/0/CHNCXR_0008_0.png',
  'dataset/0/CHNCXR_0009_0.png',
  'dataset/0/CHNCXR_0010_0.png',
  'dataset/0/CHNCXR_0011_0.png',
  'dataset/0/CHNCXR_0012_0.png',
  'dataset/0/CHNCXR_0013_0.png',
  'dataset/0/CHNCXR_0014_0.png',
  'dataset/0/CHNCXR_0015_0.png',
  'dataset/0/CHNCXR_0016_0.png',
  'dataset/0/CHNCXR_0017_0.png',
  'dataset/0/CHNCXR_0018_0.png',
  'dataset/0/CHNCXR_0019_0.png',
  'dataset/0/CHNCXR_0020_0.png',
  'dataset/0/CHNCXR_0021_0.png',
  'dataset/0/CHNCXR_0022_0.png',
  'dataset/0/CHNCXR_0023_0.png',
  'dataset/0/CHNCXR_0024_0.png',
  'dataset/0/CHNCXR_0025_0.png',
  'dataset/0/CHNCXR_0026_0.png',
  'dataset/0/CHNCXR_0027_0.png',
  'dataset/0/CHNCXR_0028_0.png',
  'dataset/0/CHNCXR_0029_0.png',
  'dataset/0/CHNCXR_0030_0.png',
  'dataset

In [0]:
pos_labels=[[i,1] for i in filesP]
neg_labels=[[i,0] for i in filesN]
ds=pos_labels+neg_labels

In [0]:
np.random.seed(4200)
import pandas as pd 
df = pd.DataFrame(ds,columns =['path', 'label']) 
def Randomizing(df):
    df2 = df.reindex(np.random.permutation(df.index))
    return df2
df=Randomizing(df)

In [0]:
from sklearn.model_selection import train_test_split
dfTrain,dfTest=train_test_split(df, shuffle=False,test_size=0.2, random_state=42)

In [0]:
from fastai.vision import *
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [0]:
bs =16
sz=224
tfms = get_transforms(do_flip=True,flip_vert=True,max_lighting=0.4,max_zoom=1.1)
src = (ImageList.from_df(df=dfTrain,path='',cols='path') #get dataset from dataset
        .split_by_rand_pct(0.2) #Splitting the dataset
        .label_from_df(cols='label') #obtain labels from the level column
      )
data= (src.transform(tfms,size=224,padding_mode='zeros') #Data augmentation
        .databunch(bs=bs,num_workers=4) #DataBunch
        .normalize(imagenet_stats) #Normalize     
       )

In [0]:
# from sklearn.metrics import roc_auc_score,f1_score
# def f1_score_a(y_pred,y_true,tens=True):
# #     score=roc_auc_score(y_true,torch.sigmoid(y_pred)[:,1])
#     f1a_score=f1_score(y_true, np.round(torch.sigmoid(y_pred)[:,1]), average='macro') 
#     if tens:
# #         score=tensor(score)
#         f1a_score=tensor(f1a_score)
#     else:
#         f1a_score=f1a_score
#     return f1a_score


In [14]:

import torch 
import torchvision
model = torchvision.models.resnet34(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth


In [0]:
num_ftrs = model.fc.in_features
model.fc=nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(num_ftrs,2)
    
)

In [0]:
model=model.cuda()

In [0]:
from fastai.callbacks import *

learn = Learner(data, model, metrics=[error_rate, accuracy])

In [0]:
from fastai.callbacks import *
learn.fit(25,lr=0.00001)

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.759055,0.731475,0.447619,0.552381,01:25
1,0.692674,0.553730,0.257143,0.742857,01:17
2,0.643936,0.507277,0.238095,0.761905,01:16
3,0.624819,0.471654,0.209524,0.790476,01:16
4,0.578839,0.459925,0.209524,0.790476,01:16
5,0.540562,0.492625,0.238095,0.761905,01:16
6,0.503699,0.504091,0.219048,0.780952,01:16
7,0.482803,0.515012,0.228571,0.771429,01:17
8,0.463435,0.417211,0.200000,0.800000,01:17


In [0]:
learn.unfreeze() 

In [0]:
learn.fit(25,lr=0.00001)

In [0]:
bs =16
sz=224
tfms = get_transforms(do_flip=True,flip_vert=True,max_lighting=0.4,max_zoom=1.1)
src = (ImageList.from_df(df=dfTest,path='',cols='path') #get dataset from dataset
        .split_by_rand_pct(0.0) #Splitting the dataset
        .label_from_df(cols='label') #obtain labels from the level column
      )
data= (src.transform(tfms,size=224,padding_mode='zeros') #Data augmentation
        .databunch(bs=bs,num_workers=4) #DataBunch
        .normalize(imagenet_stats) #Normalize     
       )

In [0]:
learn.data.valid_dl = data.train_dl

In [0]:

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()
conf=interp.confusion_matrix()
TrueNagitive=conf[0][0]
FalseNegative=conf[0][1]
TruePositive=conf[1][1]
FalsePositive=conf[1][0]
recal=TruePositive/(TruePositive+FalseNegative)
precision=TruePositive/(TruePositive+FalsePositive)
print("Precision of Model =",precision,"Recall of Model ", recal)
f1=2*((precision*recal)/(precision+recal))
print('F1 Score of Model =',f1)